In [ ]:
from pydantic import BaseModel, Field, field_validator, ConfigDict, model_validator, computed_field, EmailStr
from typing import List, Dict, Optional
from datetime import datetime
from fastapi import FastAPI, Depends

In [ ]:
# Foundations

class User(BaseModel):
    id: int
    name: str
    is_active: bool

input_data = {'id': 101, 'name': 'ChaiCode', 'is_active': True}

user = User(**input_data) # ** is used for expansion of the input_data

print(user)

In [ ]:
# Fields Validations

class Cart(BaseModel):
    user_id: int
    items: List[str]
    quantities: Dict[str, int]

class BlogPost(BaseModel):
    title: str
    content: str
    image_url: Optional[str] = None

class Employee(BaseModel):
    id: int
    name: str = Field(...,
                      min_length=3,
                      max_length=50,
                      description="Employee Name",
                      examples=["John Doe"]) # ... means its a required field
    department: Optional[str] = 'General'
    salary: float = Field(..., ge=10000)

In [ ]:
# Model Behaviour

class User(BaseModel):
    username: str

    @field_validator('username')
    def username_length(cls, val): # custom validation that the data is of right datatype
        if len(val) < 4:
            raise ValueError("Username must be atleast 4 characters")
        return val

class SignUpData(BaseModel):
    password: str
    confirm_password: str

    @model_validator(mode='after') # check if the value contained is ok or not,
    def password_match(cls, values):
        if values.password != values.confirm_password:
            raise ValueError('Password do not match')
        return values

class Product(BaseModel):
    price: float
    quantity: int

    @computed_field # creates a new property that can be easily accessed and validated
    @property # it will create a property
    def total_price(self) -> float: # check function name is the name of the property
        return self.price * self.quantity

In [ ]:
# nested models and a small mistake

class Address(BaseModel):
    street: str
    city: str
    postal_code: str

class User(BaseModel):
    id: int
    name: str
    address: Address

class Comment(BaseModel):
    id: int
    content: str
    replies: Optional[List["Comment"]] = (
        None  # self referencing by puttin same class into it
    )

Comment.model_rebuild()  # if you are doing forward referencing like we did in the last line above

address = Address(street="123 main st", city="Lahore", postal_code="54000")

user = User(id=1, name="Jamal", address=address)

comment = Comment(
    id=1,
    content="First Comment",
    replies=[
        Comment(id=2, content="reply1"),
        Comment(id=3, content="reply3"),
    ],
)

In [ ]:
# Serialization

class Address(BaseModel):
    street: str
    city: str
    zipcode: str

class User(BaseModel):
    id: int
    name: str
    email: str
    is_active: bool = True
    createdAt: datetime
    address: Address
    tags: List[str] = []

    model_config = ConfigDict(
        json_encoders={datetime: lambda v: v.strftime("%d-%m-%Y %H:%M:%S")}
    )

user = User(
    id=1,
    name="Jamal",
    email="jamal@gmail.com",
    createdAt=datetime(2024, 3, 15, 14, 30),
    address=Address(street="something", city="Jaipur", zipcode="001144"),
    is_active=False,
    tags=["premium", "subscriber"],
)

print(user.model_dump())
print("-----------------------\n")
print(user.model_dump_json())

In [ ]:
app = FastAPI()

class UserSignup(BaseModel):
    username = str
    email = EmailStr
    password = str

class Settings(BaseModel):
    app_name: str = "Chai App"
    admin_email: str = "Admin@chai.com"

def get_settings():
    return Settings()

@app.post('/signup')
def signu(user: UserSignup):
    return {'message': f'User {user.username} signed up successfully'}

@app.get('/settings')
def get_settings_endpoint(settings: Settings = Depends(get_settings)):
    return settings

# dependency injection is separation of object creation and object usage (Object can be any reference, like a function, class obj, etc...)